In [ ]:
mport torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalising based on ImageNet values
])

In [ ]:
batch_size = 16

train_data_dir = '../CURE-TSR/Train'
val_data_dir = '../CURE-TSR/Test'

train_dataset = datasets.ImageFolder(root=train_data_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_data_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Loading the pre-trained VGG19 model
model = models.vgg19(weights='DEFAULT') 

# Freezing the feature extraction layers
for param in model.parameters():
    param.requires_grad = False

num_classes = 5  # 5 categories: Rain, Snow, Shadow, Haze, Lens Blur
model.classifier[6] = nn.Linear(4096, num_classes)

# Moving the model to mps
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = model.to(device)

# Loss function (CrossEntropyLoss for multi-class classification)
criterion = nn.CrossEntropyLoss()

# The optimizer (only for the classifier part of the network)
optimizer = optim.Adam(model.classifier.parameters(), lr=0.0001)

# Define a learning rate scheduler to reduce learning rate if validation loss plateaus
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

In [ ]:

num_epochs = 10


for epoch in range(num_epochs):
    model.train()  
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        
        optimizer.zero_grad()

        
        outputs = model(images)
        loss = criterion(outputs, labels)

        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()


    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {running_loss/len(train_loader):.4f}")


    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass for validation
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    print(f"Validation Loss: {val_loss/len(val_loader):.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Adjust learning rate if validation loss plateaus
    scheduler.step(val_loss)

torch.save(model.state_dict(), "model_vgg19_adverse_env.pth")

print("Training complete. Model saved.")